In [80]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import torch.nn as nn


In [64]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [3]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev,0)

In [9]:
xprev.shape

torch.Size([8, 2])

In [8]:
xbow.shape

torch.Size([4, 8, 2])

In [26]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow2,xbow)

True

In [28]:
x.shape

torch.Size([4, 8, 2])

In [29]:
wei.sum(1, keepdim=True).shape

torch.Size([8, 1])

In [ ]:
tril = torch.tril(torch.ones(T,T)) 
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
xbow3 = wei @ x #(T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow,xbow3)

True

In [ ]:
head_size=16
key = f.linear(C,head_size, bias=False)
query = f.linear(C,head_size, bias=False)
value = f.linear(C,head_size, bias=False)

k = key(x)
q = query(x)
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

In [61]:
xbow3.shape

torch.Size([4, 8, 32])

In [ ]:
# version 4: self-attention!
# head_size=16
key = nn.Linear(C,head_size, bias=False)
query = nn.Linear(C,head_size, bias=False)
value = nn.Linear(C,head_size, bias=False)

k = key(x)
q = query(x)
v = value(x)

wei = q @ k.transpose(-2,-1)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

out = wei @ v



k.shape, q.shape, wei.shape, tril.shape, out.shape

(torch.Size([4, 8, 16]),
 torch.Size([4, 8, 16]),
 torch.Size([4, 8, 8]),
 torch.Size([8, 8]),
 torch.Size([4, 8, 16]))

In [103]:
print(wei[0].sum(dim=1))  # Sum across keys for each query at batch 0

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7949, 0.2051, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2588, 0.2702, 0.4710, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3022, 0.1535, 0.5284, 0.0159, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0973, 0.1471, 0.6608, 0.0414, 0.0533, 0.0000, 0.0000, 0.0000],
        [0.1275, 0.0246, 0.2568, 0.0038, 0.5577, 0.0295, 0.0000, 0.0000],
        [0.0327, 0.0201, 0.0817, 0.0907, 0.3184, 0.1254, 0.3310, 0.0000],
        [0.0465, 0.0457, 0.0096, 0.5060, 0.0203, 0.0600, 0.1985, 0.1136]],
       grad_fn=<SelectBackward0>)

In [117]:
vocab_size = 26
n_embd = 32
block_size = 8


token_embedding_table = nn.Embedding(vocab_size, n_embd)
position_embedding_table = nn.Embedding(block_size, n_embd)

In [120]:
x = torch.randn(B,T)

In [121]:
tok_embd = token_embedding_table(x)  # (B, T, n_embd)
# pos_embd = position_embedding_table(torch.arange(T))  # (T, n_embd)
# x = tok_embd + pos_embd

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)